In [157]:
import os
print(os.getcwd())
print('Inicio del Jupyter File.')

c:\Users\andre\OneDrive\Programación-Datos\Proyectos para portfolio-ideas\ADR Index para acciones argentinas
Inicio del Jupyter File.


# ArgDR Index para monitorear el interés internacional en la economía argentina

*Por Andrés Fernández Apenburg*

*Licenciado en Economía, Universidad de Buenos Aires*

## Introducción
Existe una clase de activos financieros llamados ADR (del inglés American Drawing Right). Un ADR es un certificado de titularidad, por parte de un residente estadounidense, de una acción de una empresa cuyas acciones cotizan en un mercado fuera de EEUU. En el caso argentino, se trata de certificados de titularidad de acciones que cotizan en la Bolsa de Comercio de Buenos Aires (BCBA). La compraventa de los ADR argentinos tiene lugar en la Bolsa de Comercio de Nueva York (NYSE) y los valores se expresan en dólares estadounidenses. Existen ADR para múltiples empresas importantes de Argentina, como YPF, Grupo Galicia, e IRSA. 

La cotización de los ADR argentinos es seguida atentamente por los medios de comunicación financieros del país y el exterior, ya que provee una valiosa señal de las expectativas futuras de generación de dólares por parte del sector privado argentino. Sin embargo, no se cuenta con un índice que permita sintetizar el conjunto de los precios, es decir, una única variable numérica, cuya variación día a día refleje la tendencia general de los precios de los múltiples ADR argentinos. 


## La solución propuesta


### Qué buscamos en un índice
Las propiedades deseables de un índice incluyen:
1. Que varíe su valor si varía el estado del fenómeno subyacente a medir (en este caso, las tendencias *generales* del valor de los ADR en su conjunto)
2. Que no se vea afectado por *outliers* (en este caso, firmas pequeñas que aumenten su valor accionario en proporciones mucho mayores que la tendencia general en el resto de las firmas)



### Definición del índice
El Índice ArgDR está definido como un promedio ponderado de todos los precios de ADR argentinos, donde los ponderadores sugeridos son la capitalización de mercado de cada firma como porcentaje de la capitalización bursátil colectiva del total de las firmas incluidas. 
Formalmente, dada una cantidad $N$ de ADR argentinos cotizando en el instante o periodo $t$ (por ejemplo, el inicio o cierre de la jornada, o las 12 del mediodía, o un cierto período sobre el cual se toman valores promedio para todos los activos), el índice queda definido como: 

$$ 
ArgDR_{t} = \sum_{i=1}^{N} A_{i, t} \cdot P_{i, t}
$$

Donde $A_{i,t}$ es la cotización en dólares del ADR $i$ en $t$, y $P_{i,t}$ es el ponderador del ADR $i$ en la jornada $t$, definido como:

$$
P_{i, t}=\frac{C_{i, t}}{\sum_{i}^{N}C_{i, t}}
$$

Donde ${C_{i, t}}$ es la capitalización bursátil de la empresa $i$, es decir, su número total de acciones multiplicado por $A_{i, t}$.

Bajo esta definición se cumple que $\sum_{i}^{N}P_{i,t} = 1$, para todo $t$, por lo que se trata de un conjunto válido de ponderadores.

En la práctica, el ponderador se puede tomar como fijo a lo largo de múltiples $t$ (índice tipo Laspeyres), utilizando, por ejemplo, las capitalizaciones bursátiles al cierre del trimestre anterior para calcular los ponderadores, de forma tal que resulta opcional su actualización diaria. 


### Por qué nuestro índice satisface las propiedades deseadas
1. Al tratarse de un promedio ponderado de precios de acciones individuales, si suben todas las acciones, necesariamente debe subir el valor del índice
2. Si una firma es más relevante en el mercado (medido en términos de la capitalización bursátil, esto es, del valor implícito de la compañía en función de la cantidad de acciones y su precio), el índice responderá con un mayor cambio porcentual ante la misma variación porcentual en el valor de dicho activo en comparación con una firma menos relevante, porque el valor de su ponderador será mayor.

# Obtención de los datos

## Web scraping desde investing.com


Descargamos un archivo CSV con la tabla que tiene investing.com mostrando las cotizaciones de ADR argentinos, el cual se puede descargar desde [esta](https://www.investing.com/equities/argentina-adrs) web.

Decidimos remover MercadoLibre de nuestro índice, ya que no es un ADR propiamente dicho (no cotiza en la BCBA salvo como CEDEAR) y sus resultados financieros dependen principalmente de sus actividades fuera de la Argentina, factores que las distinguen de todos los demás activos considerados.

In [158]:
import pandas as pd


#Cargamos distintas fuentes de datos para evaluar robustez

ADRs_20240102 = 'Data\Arg_ADRs_2024_01_02.csv'
ADRs_20240221 = 'Data\Argentina_ADRs.csv'
ADRs_20240223 = 'Data\Arg_ADRs_2024_02_23.csv'



def generar_tabla(csvfile):
    
    import pandas as pd

    tablaADR = pd.read_csv(csvfile)

    tablaADR = pd.DataFrame(tablaADR)

    tablaADR.set_index('Name', inplace = True)

    #print(tablaADR.index.values)

    # Removemos MercadoLibre (MELI) de la tabla y creamos una 'tablaADR_sinMELI'

    tablaADR_sinMELI = tablaADR.drop(['MercadoLibre'])

    #print(tablaADR_sinMELI.index.values)

    return tablaADR_sinMELI

generar_tabla(ADRs_20240221)


,Last,High,Low,Chg.,Chg. %,Vol.,Time
Name,,,,,,,
YPF Sociedad Anonima,16.960,17.400,16.800,0.250,1.50%,3.03M,15:59:59
Grupo Supervielle,4.275,4.370,4.175,-0.035,-0.81%,668.7K,16:00:00
Loma Negra ADR,6.490,6.530,6.280,0.090,1.41%,543.86K,16:00:00
Grupo Financiero Galicia ADR,20.780,20.860,20.348,0.010,0.05%,516.67K,15:59:59
BBVA Argentina,5.720,5.830,5.650,-0.100,-1.72%,491.08K,15:59:59
Banco Macro B ADR,33.62,33.68,31.86,1.140,3.51%,380.2K,15:59:59
Central Puerto,8.380,8.620,8.272,-0.040,-0.48%,300.34K,16:00:00
Transportadora Gas ADR,13.200,13.592,13.010,-0.010,-0.08%,170.73K,15:59:59
Cresud SACIF,8.270,8.470,8.240,-0.130,-1.55%,147.12K,15:59:59


In [159]:
cap_bursatiles = {
    'YPF Sociedad Anonima' : 6669706272,
    'Grupo Supervielle' : 379972012,
    'Grupo Financiero Galicia ADR' : 3064410163,
    'BBVA Argentina' : 1169255067,
    'Banco Macro B ADR' : 2153544365,
    'Telecom Argentina ADR' : 3303757407,
    'Cresud SACIF' : 495623283,
    'Central Puerto' : 1268750654,
    'Pampa Energia ADR' : 2326836834,
    'Loma Negra ADR' : 757609060,
    'Despegar.com' : 564813000,
    'IRSA ADR' : 564813000,
    'Transportadora Gas ADR' : 1981267110,
    'Bioceres Crop' : 806938546,
    'Edenor ADR' : 858866207}

suma_cap = sum(cap_bursatiles.values())

print(f'Las capitalizaciones suman {suma_cap}')

firmas_y_pond = []

for i in cap_bursatiles:
    firmas_y_pond.append([i, (cap_bursatiles[i]/suma_cap)])

ponderadores = dict(firmas_y_pond)


print()
print()
print()

print('PONDERADORES:')
for i in ponderadores:
    print(i, ':', round(ponderadores[i], 3))


print()
print()


print(f'Los ponderadores suman {round(sum(ponderadores.values()), 2)}, como corresponde.')



Las capitalizaciones suman 26366162980



PONDERADORES:
YPF Sociedad Anonima : 0.253
Grupo Supervielle : 0.014
Grupo Financiero Galicia ADR : 0.116
BBVA Argentina : 0.044
Banco Macro B ADR : 0.082
Telecom Argentina ADR : 0.125
Cresud SACIF : 0.019
Central Puerto : 0.048
Pampa Energia ADR : 0.088
Loma Negra ADR : 0.029
Despegar.com : 0.021
IRSA ADR : 0.021
Transportadora Gas ADR : 0.075
Bioceres Crop : 0.031
Edenor ADR : 0.033


Los ponderadores suman 1.0, como corresponde.


# Cómputo del índice

Teniendo los datos de precios y los ponderadores, podemos calcular el índice ArgDR para el 21-02-2024 (ya que tenemos los datos de precios para esa fecha y los ponderadores que se reutilizan de jornada en jornada, actualizándose únicamente a mediano plazo)

In [168]:
def preciosypond(csvfile):
    generar_tabla(csvfile)
    ponderadores_df = pd.DataFrame(ponderadores.values(), index=ponderadores.keys())
    precios_y_pond = ponderadores_df.merge(generar_tabla(csvfile), on=ponderadores_df.index)
    precios_y_pond.rename(columns = {'key_0' : 'Firma', 0 : 'Ponderador'}, inplace=True)
    return precios_y_pond

'''
Podemos ver que, al fluctuar los precios de las acciones, los datos provenientes 
 del primer archivo difieren de los del segundo
'''


#print(preciosypond(ADRs_20240221)[['Firma','Last']][0:5])
preciosypond(ADRs_20240223)[['Firma','Last']]
#preciosypond(ADRs_20240102)[['Firma','Last']]

,Firma,Last
0,YPF Sociedad Anonima,18.125
1,Grupo Supervielle,4.320
2,Grupo Financiero Galicia ADR,6.241
3,BBVA Argentina,22.190
4,Banco Macro B ADR,36.31
5,Telecom Argentina ADR,6.751
6,Cresud SACIF,14.430
7,Central Puerto,8.550
8,Pampa Energia ADR,7.890
9,Loma Negra ADR,8.74


In [161]:
from datetime import datetime
import os
import time

def fechacsv(csvfile) :
    creationdate = str(datetime.fromtimestamp(os.path.getctime(csvfile)))
    return creationdate[0:16]

print(fechacsv(ADRs_20240221))

2024-02-21 18:58


In [165]:





def calculo_indice(col, csvfile):
    precios_y_pond = preciosypond(csvfile)
    vector_pond = pd.Series(precios_y_pond['Ponderador'])
    #print(vector_pond)
    vector_ult_precio = pd.Series(precios_y_pond[col], dtype='float')
    #print(vector_ult_precio)
    ArgDR_Index = vector_pond.dot(vector_ult_precio)
    print(f'El índice ArgDR ({col}) el día {fechacsv(csvfile)} vale {round(ArgDR_Index, 3)} puntos')
    return ArgDR_Index

#calculo_indice('Last', ADRs_20240221)
#calculo_indice('Low',  ADRs_20240221)
#calculo_indice('High', ADRs_20240221)



#print(generar_tabla(ADRs_20240223))
#calculo_indice('Last', ADRs_20240223)

anteayer = calculo_indice('Last', ADRs_20240221)
hoy = calculo_indice('Last', ADRs_20240223)

print(f'La variación porcentual del ArgDR Index entre el 21-02' 
      f' y el 23-02 es de {round(100*((hoy/anteayer)-1),3)}%.')



El índice ArgDR (Last) el día 2024-02-21 18:58 vale 15.153 puntos
El índice ArgDR (Last) el día 2024-02-23 15:13 vale 14.547 puntos
La variación porcentual del ArgDR Index entre el 21-02 y el 23-02 es de -4.002%.


# Serie histórica

Creamos un archivo CSV denominado 'Serie_Historica_ArgDR', y una función para agregarle registros con el valor computado del índice en la fecha correspondiente.

In [163]:
def registrar(csvfile):
    registro = {0:[fechacsv(csvfile)], 1: [calculo_indice('Last', csvfile)]}
    registro = pd.DataFrame(data=registro)
    print(registro)
    registro.to_csv('Data\Serie_Historica_ArgDR.csv', mode='a', header=False, index=False)

registrar(ADRs_20240102)



El índice ArgDR (Last) el día 2024-02-23 16:34 vale 13.237 puntos
                  0          1
0  2024-02-23 16:34  13.236798


In [50]:
'''

import pandas as pd
import requests


url_inv_adr = 'https://www.investing.com/equities/argentina-adrs'

url_puentenet_adr = 'https://www.puentenet.com/cotizaciones/acciones/'

request_headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}

response =  requests.get(url_inv_adr, headers = request_headers)

tablas_adrs = pd.read_csv(response.text, index_col = 0)

print(tablas_adrs)

'''


"\n\nimport pandas as pd\nimport requests\n\n\nurl_inv_adr = 'https://www.investing.com/equities/argentina-adrs'\n\nurl_puentenet_adr = 'https://www.puentenet.com/cotizaciones/acciones/'\n\nrequest_headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}\n\nresponse =  requests.get(url_inv_adr, headers = request_headers)\n\ntablas_adrs = pd.read_csv(response.text, index_col = 0)\n\nprint(tablas_adrs)\n\n"